In [1]:
import pandas as pd
import numpy as np
import yaml
from pathlib import Path

from utils import nfl_weeks_pull, game_keys_pull
from yahoo_query import league_season_data

dates = [
    # np.datetime64("2021-09-28"), 
    # np.datetime64("2020-09-28"), 
    # np.datetime64("2019-09-28"), 
    np.datetime64("2018-09-28"), 
    np.datetime64("2017-09-28"), 
    np.datetime64("2016-09-28"), 
    np.datetime64("2015-09-28"), 
    np.datetime64("2014-09-28"), 
    np.datetime64("2013-09-28"), 
    np.datetime64("2012-09-28")
]

PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
NFL_WEEKS = nfl_weeks_pull()
GAME_KEYS = game_keys_pull(first="no")
for today in dates:

    SEASON = today.astype("datetime64[Y]").item().year
    LEAGUE_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["league_ID"].values[0]
    GAME_ID = GAME_KEYS[GAME_KEYS["season"] == SEASON]["game_id"].values[0]
    nfl_weeks_list = list(NFL_WEEKS["week"][NFL_WEEKS["game_id"] == GAME_ID])

    try:
        with open(PATH) as file:
            credentials = yaml.load(file, Loader=yaml.FullLoader)

    except Exception as error:
        print(error)

    CONSUMER_KEY = credentials["YFPY_CONSUMER_KEY"]
    CONSUMER_SECRET = credentials["YFPY_CONSUMER_SECRET"]

    league = league_season_data(
        auth_dir=PATH.parent,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )

    # league.all_game_keys()
    # league.all_nfl_weeks()

    # league.metadata(first_time=first_time)
    # league.set_roster_pos_stat_cat(first_time=first_time)
    # league.draft_results(first_time=first_time)
    # league.teams_and_standings(first_time="no")

    # league.matchups_by_week_regseason(first_time='yes', nfl_week=1)
    for week in nfl_weeks_list[:-1]:
        league.matchups_by_week_regseason(first_time='no', nfl_week=week)

    # league.players_list(first_time="no")

    # league.team_roster_by_week(first_time='yes', nfl_week=1)
    for week in nfl_weeks_list[:-1]:
        league.team_roster_by_week(first_time='no', nfl_week=week)

    # league.team_points_by_week( first_time='yes', nfl_week=1)
    for week in nfl_weeks_list[:-1]:
        league.team_points_by_week( first_time='no', nfl_week=week)

[2022-07-16 14:32:08,393 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fe4b4b3ae80; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=prod'', closed: 0>
[2022-07-16 14:32:08,395 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
[2022-07-16 14:32:08,395 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 14:32:08,396 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
[2022-07-16 14:32:08,397 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14.4 (Ubuntu 14.4-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)
[2022-07-16 14:32:08,397 INFO] [sqlalchemy.engine.Engine.base._execute_context] select current_schema()
[2022-07-16 14:32:08,398 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 14:32:08,398 DEBUG] [sqlalche

Successfully pulled: SELECT * FROM prod.nflweeks
Successfully pulled: SELECT * FROM prod.gamekeys


[2022-07-16 14:32:09,549 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fe4b4b3bc40; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-07-16 14:32:09,550 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
[2022-07-16 14:32:09,551 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 14:32:09,551 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
[2022-07-16 14:32:09,552 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14.4 (Ubuntu 14.4-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)
[2022-07-16 14:32:09,552 INFO] [sqlalchemy.engine.Engine.base._execute_context] select current_schema()
[2022-07-16 14:32:09,553 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 14:32:09,553 DEBUG] [sqlalchem

Successfully pulled: SELECT * FROM dev.regseasonmatchups WHERE (game_id != '406' AND week != '1')
Upload successful: regseasonmatchups


[2022-07-16 14:32:10,538 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fe4b4b3afc0; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-07-16 14:32:10,540 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
[2022-07-16 14:32:10,540 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 14:32:10,541 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
[2022-07-16 14:32:10,542 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14.4 (Ubuntu 14.4-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)
[2022-07-16 14:32:10,542 INFO] [sqlalchemy.engine.Engine.base._execute_context] select current_schema()
[2022-07-16 14:32:10,543 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 14:32:10,543 DEBUG] [sqlalchem

...

[2022-07-16 15:31:03,550 DEBUG] [sqlalchemy.pool.impl.QueuePool.base.__connect] Created new connection <connection object at 0x7fe4b3b69a80; dsn: 'user=cuddebtj password=xxx dbname=MenOfMadison host=localhost options='-c search_path=dev'', closed: 0>
[2022-07-16 15:31:03,552 INFO] [sqlalchemy.engine.Engine.base._execute_context] select pg_catalog.version()
[2022-07-16 15:31:03,552 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 15:31:03,553 DEBUG] [sqlalchemy.engine.Engine.cursor._init_metadata] Col ('version',)
[2022-07-16 15:31:03,553 DEBUG] [sqlalchemy.engine.Engine.cursor.log_row] Row ('PostgreSQL 14.4 (Ubuntu 14.4-0ubuntu0.22.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 11.2.0-19ubuntu1) 11.2.0, 64-bit',)
[2022-07-16 15:31:03,554 INFO] [sqlalchemy.engine.Engine.base._execute_context] select current_schema()
[2022-07-16 15:31:03,554 INFO] [sqlalchemy.engine.Engine.base._execute_context] [raw sql] {}
[2022-07-16 15:31:03,555 DEBUG] [sqlalchem

Successfully pulled: SELECT * FROM dev.weeklyteampoints WHERE (game_id != '380' AND week != '12')
Upload successful: weeklyteampoints
Successfully pulled: SELECT team_id FROM dev.leagueteams WHERE game_id = '380'


2022-07-16 15:31:28.581 - ERROR - query.py - yfpy.query:197 - Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/380/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


Error, sleeping for 30 min before retrying.
Attempt to retrieve data at URL https://fantasysports.yahooapis.com/fantasy/v2/game/380/metadata?format=json failed with error: "Please provide valid credentials. OAuth oauth_problem="token_expired", realm="yahooapis.com""


KeyboardInterrupt: 